In [1]:
import whisper

print(whisper.__version__)

20240930


In [1]:
import ffmpeg


# 提取音频并保存为 WAV 格式
ffmpeg.input(r"E:\OneDrive - MSFT\.master_data\24-25ws\SIL\interview\Heizmann\Heizmann_Interview record.mkv").output(r"E:\OneDrive - MSFT\.master_data\24-25ws\SIL\interview\Heizmann\extracted_audio2.wav", audio_bitrate='128k').run()



(None, None)

In [1]:
from pydub import AudioSegment
import subprocess
import os

input_path = r"E:\OneDrive - MSFT\.master_data\24-25ws\SIL\interview\trans\Hans.mkv"
output_path = r"E:\OneDrive - MSFT\.master_data\24-25ws\SIL\interview\trans\Hans.mp3"


# 调用 FFmpeg 提取音频
subprocess.call(["ffmpeg", "-i", input_path, "temp_audio.wav"])

# 使用 pydub 处理提取的音频
audio = AudioSegment.from_file("temp_audio.wav", format="wav")

# 保存为 MP3 格式
audio.export(output_path, format="mp3")

os.remove("temp_audio.wav")


In [3]:
import torch
torch.cuda.empty_cache()


In [5]:
import ffmpeg
input_path = r"E:\OneDrive - MSFT\.master_data\24-25ws\SIL\interview\trans\Julia Anken.m4a"
output_path = r"E:\OneDrive - MSFT\.master_data\24-25ws\SIL\interview\trans\Julia Anken.mp3"

# 提取音频并保存为 WAV 格式
ffmpeg.input(input_path).output(output_path, audio_bitrate='128k').run()

from pydub import AudioSegment
import subprocess
import os

# 调用 FFmpeg 提取音频
subprocess.call(["ffmpeg", "-i", input_path, "temp_audio.wav"])

# 使用 pydub 处理提取的音频
audio = AudioSegment.from_file("temp_audio.wav", format="wav")

# 保存为 MP3 格式
audio.export(output_path, format="mp3")

os.remove("temp_audio.wav")

In [1]:
import torch
torch.cuda.empty_cache()

In [6]:
import whisper
import torch
import sys
from tqdm import tqdm  # 导入 tqdm

# 检查是否有可用的 GPU
device = "cuda" if torch.cuda.is_available() else None

# 如果没有 GPU，抛出错误并中止
if device is None:
    print("错误：没有可用的 GPU。程序将中止。")
    sys.exit(1)

# 加载 Whisper 模型并启用 CUDA 加速
model = whisper.load_model("base").to(device)  # 使用 'large' 模型，精度更高

###############################
# 路径设置
###############################
# 输入音频文件路径# 输出文件路径
# audio_path = r"E:\OneDrive - MSFT\.master_data\24-25ws\SIL\interview\tourism\tourism.mp3"  # 替换为你的音频文件路径
# output_txt_path = r"E:\OneDrive - MSFT\.master_data\24-25ws\SIL\interview\tourism\transcription_result.txt"  # 替换为你想保存的输出文件路径
audio_path = r"E:\OneDrive - MSFT\.master_data\24-25ws\SIL\interview\trans\Hans.mp3"  # 替换为你的音频文件路径
output_txt_path = r"E:\OneDrive - MSFT\.master_data\24-25ws\SIL\interview\trans\Hans.txt"  # 替换为你想保存的输出文件路径

###############################
# 处理音频并获取时长
###############################
# 使用 Whisper 加载音频文件
audio = whisper.load_audio(audio_path)

# Whisper 默认采样率
sample_rate = 16000

# 计算音频文件的时长（单位：秒）
duration = len(audio) / sample_rate

###############################
# 转录音频并显示进度条
###############################
# 自定义带进度条的转录函数
def transcribe_with_progress(model, audio_path, pbar):
    # 清理 CUDA 缓存
    torch.cuda.empty_cache()

    # 启动 Whisper 的转录，禁用内置的默认进度条
    result = model.transcribe(audio_path, language="en")
    for segment in result["segments"]:
        # 每处理一个段落，更新进度条
        pbar.update(segment["end"] - pbar.n)  # 以段落结束时间为更新基准
    return result

# 创建进度条并执行转录
with tqdm(total=duration, unit="s", desc="Transcribing", ncols=100) as pbar:
    result = transcribe_with_progress(model, audio_path, pbar)

###############################
# 保存转录结果
###############################
# 将结果保存为带时间戳和分段的文本
with open(output_txt_path, "w", encoding="utf-8") as f:
    for segment in result["segments"]:
        start = segment["start"]
        end = segment["end"]
        text = segment["text"]
        # 格式化输出时间戳和文字
        f.write(f"[{start:.2f}s - {end:.2f}s] {text}\n")

print(f"转录结果已保存至 {output_txt_path}")

# 清理 CUDA 缓存
torch.cuda.empty_cache()


Transcribing: 100%|███████████████████████████████████████▉| 3515.8/3516.469 [01:03<00:00, 55.28s/s]

转录结果已保存至 E:\OneDrive - MSFT\.master_data\24-25ws\SIL\interview\trans\transcription_result_Julia Anken.txt


In [ ]:
import whisper
import torch
import sys
from tqdm import tqdm  # 导入 tqdm
from pyannote.audio import Pipeline  # 用于说话人分离
from pyannote.core import Segment

# 检查是否有可用的 GPU
device = "cuda" if torch.cuda.is_available() else None

# 如果没有 GPU，抛出错误并中止
if device is None:
    print("错误：没有可用的 GPU。程序将中止。")
    sys.exit(1)

# 加载 Whisper 模型并启用 CUDA 加速
model = whisper.load_model("large").to(device)  # 使用 'large' 模型，支持详细标点

###############################
# 路径设置
###############################
# 输入音频文件路径
audio_path = r"E:\OneDrive - MSFT\.master_data\24-25ws\SIL\interview\tourism\tourism.mp3"  # 替换为你的音频文件路径
# 输出文件路径
output_txt_path = r"E:\OneDrive - MSFT\.master_data\24-25ws\SIL\interview\tourism\transcription_result_with_speakers.txt"  # 替换为你想保存的输出文件路径

###############################
# 使用 pyannote 进行说话人分离
###############################
# 使用 pyannote.audio 的预训练管道
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization", use_auth_token="你的HuggingFace令牌")  # 需要 Hugging Face 账号

# 运行说话人分离
diarization = pipeline(audio_path)

###############################
# 使用 Whisper 转录音频
###############################
# 加载音频并计算时长
audio = whisper.load_audio(audio_path)
sample_rate = 16000
duration = len(audio) / sample_rate

# 自定义带进度条的转录函数
def transcribe_with_progress(model, audio_path, pbar):
    # 清理 CUDA 缓存
    torch.cuda.empty_cache()

    # 启动 Whisper 的转录，禁用内置的默认进度条
    result = model.transcribe(audio_path, language="en", progress_bar=False)
    for segment in result["segments"]:
        # 每处理一个段落，更新进度条
        pbar.update(segment["end"] - pbar.n)  # 以段落结束时间为更新基准
    return result

# 创建进度条并执行转录
with tqdm(total=duration, unit="s", desc="Transcribing", ncols=100) as pbar:
    result = transcribe_with_progress(model, audio_path, pbar)

###############################
# 合并转录和说话人分离结果
###############################
# 将每段转录结果和说话人分段对齐
def assign_speakers_to_segments(diarization, whisper_segments):
    speaker_segments = []
    for segment in whisper_segments:
        start_time = segment["start"]
        end_time = segment["end"]
        text = segment["text"]

        # 查找说话人信息
        for turn, _, speaker in diarization.itertracks(yield_label=True):
            if turn.intersects(Segment(start_time, end_time)):
                speaker_segments.append((speaker, start_time, end_time, text))
                break
    return speaker_segments

# 获取合并结果
merged_segments = assign_speakers_to_segments(diarization, result["segments"])

###############################
# 保存结果到文件
###############################
# 将结果保存为带详细标点、时间戳、分段和说话人标注的文本
with open(output_txt_path, "w", encoding="utf-8") as f:
    for speaker, start, end, text in merged_segments:
        f.write(f"[{start:.2f}s - {end:.2f}s] {speaker}: {text}\n")

print(f"转录结果已保存至 {output_txt_path}")

# 清理 CUDA 缓存
torch.cuda.empty_cache()
